<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/fine-tuning/Tech3_Finetuning_com_LoRA_e_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning utilizando Tinyllama com Unsloth

### Instalação da Biblioteca `unsloth`

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

### Carregamento do modelo

Este código utiliza a biblioteca `unsloth` para carregar um modelo de linguagem quantizado em 4 bits, configurado com um comprimento máximo de sequência de 2048 tokens. O modelo é inicialmente carregado com um nome predefinido e pode ser alterado para uma versão customizada após o primeiro treino.

**Principais configurações:**
- `max_seq_length`: Comprimento máximo da sequência (2048 tokens).
- `dtype`: Detecção automática do tipo de dado. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
- `load_in_4bit`: Quantização em 4 bits ativada para economia de memória.
- `new_model_name`: Nome do modelo personalizado a ser usado após o primeiro treino.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

#new_model_name = unsloth/tinyllama-chat-bnb-4bit # Alterar após o 1º treino
new_model_name = "rafaelpivetta/tinyllama-chat-bnb-4bit-g19"
data_set_name = "LuizfvFonseca/trn_limpo_parte_X_de_70"

# Após o 1º treino, alterar o model para rafaelpivetta/tinyllama-chat-bnb-4bit-g19
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/tinyllama-chat-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# tokenizer.pad_token = tokenizer.eos_token # pad sequences - não é indicado que o pad_token seja o mesmo que o eos_token
# tokenizer.padding_side = 'right' # right pad sequences

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/50.5M [00:00<?, ?B/s]

Unsloth 2024.9.post1 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


### Criação do alpaca_prompt para cada linha do dataset, coluna 'text', que será usado no SFTTrainer

O código abaixo formata um conjunto de dados para treinamento de um modelo de linguagem usando um template de prompt no estilo Alpaca. O processo envolve a criação de instruções personalizadas para cada exemplo no conjunto de dados e a formatação dessas instruções em um prompt específico.

**Passos do Código:**
1. **Template `alpaca_prompt`:** Define um formato de prompt que inclui uma instrução (`Instruction`) e uma resposta (`Response`), com um token de fim de sequência (EOS_TOKEN).
2. **Função `formatting_prompts_func`:** Recebe as instruções e os conteúdos do dataset, aplica o template `alpaca_prompt`, e adiciona o token de fim de sequência, criando uma nova coluna chamada `text`.
3. **Carregamento do Dataset:** Carrega um dataset especificado pela variável `data_set_name`, utilizando a divisão de treinamento (`train`).
4. **Função `add_instruction_column`:** Cria uma nova coluna `instruction` no dataset, que consiste em uma frase que pede para criar uma descrição do produto baseado no título de cada item do dataset.
5. **Mapeamento das Funções:** Aplica as funções `add_instruction_column` e `formatting_prompts_func` ao dataset, gerando as instruções personalizadas e os textos formatados.
6. **Exemplo de Texto Formatado:** Exibe um exemplo de prompt formatado do dataset.

In [3]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # EOS_TOKEN - token de fim de sequência

# Função responsável pela criação do alpaca_prompt de cada linha e criação da coluna 'text' que será usada no treino 
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    contents    = examples["content"]
    texts = []
    for instruction, content in zip(instructions, contents):
        text = alpaca_prompt.format(instruction, content) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(data_set_name, split = "train")

# Função responsável por concatenar 'Describe the product xxxx' com o título e criar uma nova coluna instruction
def add_instruction_column(example):
    example["instruction"] =  f"Create a description for the following product: {example['title']}"
    return example

# Aplica a função add_instruction_column ao dataset
dataset = dataset.map(add_instruction_column)

dataset = dataset.map(formatting_prompts_func, batched = True,)

alpaca_prompt_text = dataset['text'][3]
print(alpaca_prompt_text)

trn_limpo_2_de_70.json:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a description for the following product: Design-For-Test For Digital IC's and Embedded Core Systems

### Response:
PrefaceThis book is made primarily for design engineers and managers, and for test and design-for-test engineers and managers. It can also be used for students of digital design and test, as well. The purpose of this book is to introduce the basic concepts of test and design-for-test (DFT), and to then address the application of these concepts with an eye toward the trade-offs of the engineering budgets (silicon area, operating frequency target, power consumption, etc.), the business drivers, and the cost factors.Currently, some very good test and DFT texts are available. However, many of them are from an academic focus. In my years of being part of the integrated circuit design community, I have had to train many IC designers and junior DFT en

### Inferência antes do treino

1. **Habilita Inferência Rápida:** O método `FastLanguageModel.for_inference(model)` é usado para ativar uma inferência até 2 vezes mais rápida no modelo carregado.
2. **Tokenização dos Inputs:** A instrução é formatada com o `alpaca_prompt` para criar um prompt, que é então tokenizado e preparado para ser processado pelo modelo em um dispositivo CUDA (GPU).

In [4]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Create a description for the following product: Invitation to Cryptology", # instruction
        "", # response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

print(inputs)

['<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a description for the following product: Invitation to Cryptology\n\n### Response:\n"This is a very good book. It is well written, well organized, and contains a wealth of information. It is a good reference book for the student of cryptology."--Journal of Cryptology</s>']

3. **Geração de Texto:** Geração de saída com base nos inputs, limitando a geração a 128 tokens. Várias opções, como `use_cache`, são usadas para otimizar a geração.

In [ ]:
outputs = model.generate(**inputs,
                         max_new_tokens = 128,
                         use_cache = True,
                        #  temperature=0.7,
                        #  top_p=0.9,
                        #  repetition_penalty=1.1
                         )

print(outputs)

4. **Decodificação da Saída:** Os tokens gerados são decodificados de volta para texto, resultando na resposta gerada pelo modelo para a instrução dada.

In [ ]:
tokenizer.batch_decode(outputs)

### Configuração do Modelo com PEFT (Parameter-Efficient Fine-Tuning)

Este código utiliza a função `get_peft_model` da biblioteca `FastLanguageModel` para aplicar o fine-tuning eficiente de parâmetros (PEFT) a um modelo de linguagem. As configurações incluem a aplicação da técnica LoRA (Low-Rank Adaptation) para ajustar os parâmetros de forma eficiente, utilizando menos memória e possibilitando o treinamento de lotes maiores.

**Principais Configurações:**
- **Rank `r`:** Define a dimensão do espaço de rank baixo (exemplo: 16).
- **Módulos Alvo (`target_modules`):** Lista de módulos específicos do modelo para aplicar a adaptação LoRA, como projeções de queries (`q_proj`), chaves (`k_proj`), valores (`v_proj`), entre outros.
- **`lora_alpha`:** Parâmetro de escala para o ajuste LoRA.
- **`lora_dropout`:** Probabilidade de dropout aplicada nos caminhos LoRA. Um valor de 0 é otimizado para essa configuração.
- **`bias`:** Configuração de bias, com a opção "none" sendo a mais otimizada.
- **`use_gradient_checkpointing`:** Ativado como "unsloth", que permite economizar 30% de VRAM e suportar contextos muito longos com batch sizes maiores.
- **`use_rslora`:** Controle de ativação para Rank Stabilized LoRA, uma variação da técnica.
- **`loftq_config`:** Configuração para LoftQ, que não está sendo utilizada neste exemplo.

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj","lm_head"], # Teste com todos os parâmetros
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


### Garbage Collection e Limpeza de Cache

É usado para otimizar o uso de recursos de memória durante o treinamento de modelos no Google Colab. A coleta de lixo ajuda a liberar memória ocupada por objetos que não são mais necessários, enquanto a limpeza do cache do CUDA assegura que a memória da GPU seja liberada.

In [6]:
#Garbage collection para que os recursos no colab não excedam no momento do treino
import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() #clean cache

## Configuração do SFTTrainer para Treinamento de Modelos

Este trecho configura um `SFTTrainer` (Supervised Fine-Tuning Trainer) para treinar um modelo de linguagem utilizando a biblioteca `transformers` e a biblioteca `unsloth`. O `SFTTrainer` é utilizado para realizar o fine-tuning de modelos de forma eficiente.

**Principais Componentes da Configuração:**

- **Modelo e Tokenizer:** O modelo e o tokenizer já carregados são passados para o treinador.
- **Dataset de Treinamento:** O dataset utilizado para treinamento é especificado, incluindo o campo de texto (`dataset_text_field`).
- **Max Sequence Length:** Define o comprimento máximo da sequência para o treinamento.

**Parâmetros do Treinamento:**
- **Batch Size:** `per_device_train_batch_size` é definido como 2, especificando quantos exemplos serão processados em cada passo de treinamento.
- **Gradient Accumulation:** `gradient_accumulation_steps` é definido como 16, permitindo acumular gradientes antes de realizar uma atualização do modelo.
- **Warmup Steps:** Define o número de passos de aquecimento antes de aumentar a taxa de aprendizado.
- **Max Steps:** O treinamento será limitado a 100 passos.
- **Learning Rate:** A taxa de aprendizado é definida como 2e-4.
- **Mixed Precision:** `fp16` e `bf16` são utilizados dependendo do suporte para `bfloat16`.
- **Gradient Clipping:** `max_grad_norm` é definido como 0.3 para estabilizar o treinamento.
- **Logging:** Define que as informações de log serão exibidas a cada passo.
- **Otimização:** Utiliza o otimizador "adamw_8bit" com uma taxa de decaimento de peso de 0.01.
- **Scheduler:** O tipo de agendador de taxa de aprendizado é definido como "linear".
- **Seed:** Um valor de semente para garantir a reprodutibilidade.
- **Group by Length:** Agrupa sequências de comprimentos semelhantes para melhorar a eficiência do treinamento.
- **Output Directory:** Especifica o diretório de saída para salvar os resultados.

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        max_grad_norm=0.3, #prevents the gradients from becoming too large and helps stabilize training. Gradient Clipping
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        group_by_length=True, #which will group similar-length sequences together to make training more efficient.
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/20000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### Treinamento do modelo

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 100
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
1,2.125600
2,2.087000
3,2.068800
4,2.055700
5,2.086200
6,2.065800
7,2.128400
8,2.111600
9,2.099300
10,2.066300


### Inferência após o treino

In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Create a description for the following product: Invitation to Cryptology", # instruction
        "", # response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs,
                         max_new_tokens = 128,
                         use_cache = True,
                        #  temperature=0.7,
                        #  top_p=0.9,
                        #  repetition_penalty=1.1
                         )
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a description for the following product: Invitation to Cryptology\n\n### Response:\n"This is a very good book. It is written in a clear and concise style, and the examples are very helpful. The book is well-organized and easy to read."--Journal of Cryptology</s>']

### Inferência após o treino utilizando `TextStreamer`

In [26]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Create a description for the following product: Invitation to Cryptology", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a description for the following product: Invitation to Cryptology

### Response:
"This is a very good book. It is well written, well organized, and well illustrated. It is a good introduction to cryptology for the non-specialist. It is also a good reference book for the specialist. It is a good book for the student of cryptology to have on hand."--Journal of the American Society for Information Science and Technology</s>


### Carregamento do modelo finetuned para o HuggingFace

In [10]:
from google.colab import userdata

# import gc # garbage collection
# gc.collect()
# torch.cuda.empty_cache() #clean cache

#model.save_pretrained("lora_model") # Local saving
#tokenizer.save_pretrained("lora_model")
model.push_to_hub(new_model_name, token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub(new_model_name, token = userdata.get('HF_TOKEN')) # Online saving


README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/50.5M [00:00<?, ?B/s]

Saved model to https://huggingface.co/rafaelpivetta/tinyllama-chat-bnb-4bit-g19


No files have been modified since last commit. Skipping to prevent empty commit.


### Fontes

- [Databricks Blog: Efficient Fine-Tuning with LoRA](https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms)
- [GitHub - Unsloth](https://github.com/unslothai/unsloth?tab=readme-ov-file)
